In [57]:
import org.apache.spark.sql.SparkSession

lastException: Throwable = null


In [58]:
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", "/user/itv001389/warehouse").
    enableHiveSupport.
    master("yarn").
    appName("Getting Started - Spark with Scala").
    getOrCreate

spark = org.apache.spark.sql.SparkSession@f6347e7


org.apache.spark.sql.SparkSession@f6347e7

In [59]:
spark.sql("use Raj")

[]

In [4]:
spark.sql("show tables").show(false)

+--------+------------------+-----------+
|database|tableName         |isTemporary|
+--------+------------------+-----------+
|raj     |order_immu        |false      |
|raj     |orderitem_nor     |false      |
|raj     |orderitem_orc     |false      |
|raj     |orderitem_par     |false      |
|raj     |order_items       |false      |
|raj     |order_orc         |false      |
|raj     |orders            |false      |
|raj     |order_skip_header |false      |
|raj     |order_skip_header1|false      |
|raj     |order_skip_header2|false      |
|raj     |order_skip_header3|false      |
|raj     |order_snap        |false      |
+--------+------------------+-----------+



In [5]:
spark.sql("select * from orders limit 5").show(false)

+--------+-------------------+----------------+---------------+
|order_id|Order_date         |Order_product_id|Order_status   |
+--------+-------------------+----------------+---------------+
|1       |2013-07-25 00:00:00|11599           |CLOSED         |
|2       |2013-07-25 00:00:00|256             |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00|12111           |COMPLETE       |
|4       |2013-07-25 00:00:00|8827            |CLOSED         |
|5       |2013-07-25 00:00:00|11318           |COMPLETE       |
+--------+-------------------+----------------+---------------+



In [6]:
spark.sql("select * from order_items limit 5").show(false)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|1            |1                  |957                  |1                  |299.9800           |299.9800                |
|2            |2                  |1073                 |1                  |199.9900           |199.9900                |
|3            |2                  |502                  |5                  |250.0000           |50.0000                 |
|4            |2                  |403                  |1                  |129.9900           |129.9900                |
|5            |4                  |897                  |2                  |49.9800            |24.9900                 |
+-------------+-

In [7]:
spark.sql("create table Raj.order_oi(order_id int,order_date timestamp,order_status string,order_item_id int,order_item_quantity int,order_item_product_price decimal(17,4)) row format delimited fields terminated by ',' stored as textfile location 'hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/order_oi' ")

[]

In [13]:
spark.sql("SELECT a.order_id,a.order_date,a.order_status,b.order_item_id,b.order_item_quantity,b.order_item_product_price FROM orders a JOIN order_items b ON (a.order_id = b.order_item_order_id) limit 10").show(false)

+--------+-------------------+---------------+-------------+-------------------+------------------------+
|order_id|order_date         |order_status   |order_item_id|order_item_quantity|order_item_product_price|
+--------+-------------------+---------------+-------------+-------------------+------------------------+
|1       |2013-07-25 00:00:00|CLOSED         |1            |1                  |299.9800                |
|2       |2013-07-25 00:00:00|PENDING_PAYMENT|2            |1                  |199.9900                |
|2       |2013-07-25 00:00:00|PENDING_PAYMENT|3            |5                  |50.0000                 |
|2       |2013-07-25 00:00:00|PENDING_PAYMENT|4            |1                  |129.9900                |
|4       |2013-07-25 00:00:00|CLOSED         |5            |2                  |24.9900                 |
|4       |2013-07-25 00:00:00|CLOSED         |6            |5                  |59.9900                 |
|4       |2013-07-25 00:00:00|CLOSED         |

In [21]:
spark.sql("insert into Raj.order_oi select a.order_id,a.order_date,a.order_status,b.order_item_id,b.order_item_quantity,b.order_item_quantity from orders a join order_items b on (a.order_id=b.order_item_order_id)")

[]

In [22]:
spark.sql("select * from Raj.order_oi limit 10").show(false)

+--------+-------------------+---------------+-------------+-------------------+------------------------+
|order_id|order_date         |order_status   |order_item_id|order_item_quantity|order_item_product_price|
+--------+-------------------+---------------+-------------+-------------------+------------------------+
|1       |2013-07-25 00:00:00|CLOSED         |1            |1                  |1.0000                  |
|2       |2013-07-25 00:00:00|PENDING_PAYMENT|2            |1                  |1.0000                  |
|2       |2013-07-25 00:00:00|PENDING_PAYMENT|3            |5                  |5.0000                  |
|2       |2013-07-25 00:00:00|PENDING_PAYMENT|4            |1                  |1.0000                  |
|4       |2013-07-25 00:00:00|CLOSED         |5            |2                  |2.0000                  |
|4       |2013-07-25 00:00:00|CLOSED         |6            |5                  |5.0000                  |
|4       |2013-07-25 00:00:00|CLOSED         |

In [33]:
spark.sql("create table raj.order_oi_cast1 as select cast(a.order_date as date),sum(b.order_item_quantity) from orders a join order_items b on (a.order_id=b.order_item_order_id) group by order_date ")

lastException: Throwable = null


[]

In [36]:
spark.sql("select * from raj.order_oi_cast limit 5").show(false)

+----------+------------------------+
|order_date|sum(order_item_quantity)|
+----------+------------------------+
|2014-06-11|795                     |
|2013-07-25|741                     |
|2014-01-13|974                     |
|2014-07-16|811                     |
|2014-07-02|667                     |
+----------+------------------------+



In [66]:
 spark.sql("create table raj.order_oi_sum2 as select a.order_date,a.order_status,sum(b.order_item_quantity*b.order_item_product_price) AS Total_Revenue from orders a join order_items b on (a.order_id=b.order_item_order_id) group by order_date,order_status")

lastException: Throwable = null


[]

In [67]:
spark.sql("select count(*) from  raj.order_oi_sum2 ").show()

+--------+
|count(1)|
+--------+
|    3165|
+--------+



In [68]:
spark.sql("select * from  raj.order_oi_sum2 limit 10").show()

+-------------------+---------------+-------------+
|         order_date|   order_status|Total_Revenue|
+-------------------+---------------+-------------+
|2014-02-01 00:00:00|       COMPLETE|   48115.3600|
|2014-04-04 00:00:00|         CLOSED|   10926.7800|
|2014-04-11 00:00:00|     PROCESSING|   12967.7400|
|2014-04-15 00:00:00|        ON_HOLD|    4961.3600|
|2014-04-30 00:00:00|PENDING_PAYMENT|   14399.4200|
|2014-05-08 00:00:00|         CLOSED|    4009.6400|
|2014-05-19 00:00:00|        ON_HOLD|    1219.8500|
|2014-06-23 00:00:00|     PROCESSING|    6155.3700|
|2014-07-13 00:00:00|        PENDING|   15974.4000|
|2014-07-14 00:00:00|         CLOSED|    5881.3400|
+-------------------+---------------+-------------+

